In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%load_ext autoreload
%autoreload 2
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm

from helpers import count_unique_words, count_unique_ngrams, \
            build_unique_ngrams, create_sentence_vectors, create_sentence_vectors_submission

import sys

import tensorflow as tf
from tensorflow import keras

import gensim   # Not sure whether it is better to use gensim or tensorflow :/
import logging
from gensim.models.phrases import Phrases, Phraser

import multiprocessing

from gensim.models import Word2Vec
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 

sys.path.append('../')

/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/stefano/anaconda3/envs/tf-gpu/lib/pyt

In [2]:
# For non binary models
w2v_model = Word2Vec.load("models/w2v_model_epochs_10_win_10_sg_300.model")

# For binary models
#w2v_model = gensim.models.KeyedVectors.load_word2vec_format('models/GoogleNews-vectors-negative300.bin', binary=True)

2019-12-16 11:10:52,584 : INFO : loading Word2Vec object from models/w2v_model_epochs_10_win_10_sg_300.model
2019-12-16 11:10:53,321 : INFO : loading wv recursively from models/w2v_model_epochs_10_win_10_sg_300.model.wv.* with mmap=None
2019-12-16 11:10:53,321 : INFO : loading vectors from models/w2v_model_epochs_10_win_10_sg_300.model.wv.vectors.npy with mmap=None
2019-12-16 11:10:53,430 : INFO : setting ignored attribute vectors_norm to None
2019-12-16 11:10:53,430 : INFO : loading vocabulary recursively from models/w2v_model_epochs_10_win_10_sg_300.model.vocabulary.* with mmap=None
2019-12-16 11:10:53,431 : INFO : loading trainables recursively from models/w2v_model_epochs_10_win_10_sg_300.model.trainables.* with mmap=None
2019-12-16 11:10:53,431 : INFO : loading syn1neg from models/w2v_model_epochs_10_win_10_sg_300.model.trainables.syn1neg.npy with mmap=None
2019-12-16 11:10:53,531 : INFO : setting ignored attribute cum_table to None
2019-12-16 11:10:53,532 : INFO : loaded models/w

In [3]:
len(w2v_model.wv.vocab)

452520

In [2]:
# word_vector_size = w2v_model.wv.vectors.shape[1]
word_vector_size = 300
word_vector_size

300

In [11]:
w2v_model.most_similar("keyboard")

/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('keyboar', 0.5493345260620117),
 ('adesso', 0.528719961643219),
 ('eltrex-tech', 0.5182769894599915),
 ('pz30au', 0.5138871669769287),
 ('akai', 0.5134224891662598),
 ('73p4067', 0.5131996870040894),
 ('q29', 0.5130102634429932),
 ('btkey', 0.5105351209640503),
 ('icekey', 0.5092136859893799),
 ('olikey', 0.5090413689613342)]

In [2]:
# This is unfortunately very high memory consumptive

# Test here is not the unlabelled testset, but rather the 0.8 split of the original training set

sentence_train_x = np.load("np_arrays/sentences_train_x_0_8_300.npy")
sentence_train_y = np.load("np_arrays/sentences_train_y_0_8_300.npy")

sentence_test_x = np.load("np_arrays/sentences_test_x_0_2_300.npy")
sentence_test_y = np.load("np_arrays/sentences_test_y_0_2_300.npy")

In [7]:
# Just print how many 0 we have, just to be sure that we have shuffled the dataset
y_ = np.argmax(sentence_train_y, axis=-1)
len(y_[y_==0])

999771

In [3]:
from tensorflow.keras.callbacks import ModelCheckpoint
# now perform training on the new features vectors.

# Build a "deep" neural network with 2 hidden layers. When we see that it somehow works,
# we can start doing some cross validation on it.

for layer_size in range(40, 101, 5):
    
    print("\n\nStarting with number of layers: {}".format(layer_size))
    model = keras.Sequential([
        keras.layers.InputLayer(input_shape=(sentence_train_x.shape[1],)),   # the input shape is the number of words in the bow dictionary
        keras.layers.Dense(layer_size, activation='relu'),
        keras.layers.Dense(layer_size, activation='relu'),
        keras.layers.Dense(2, activation='softmax')   # Only 0 and 1
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy']
                  )

    filepath="models/weights_{}_layers_of_size_{}_big".format(len(model.layers), layer_size)
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    model.fit(x=sentence_train_x,
              y=sentence_train_y,
              validation_data=(sentence_test_x,  sentence_test_y),
              callbacks=callbacks_list,
              epochs=50,
              use_multiprocessing=True,
              batch_size=128)
        




Starting with number of layers: 40
Instructions for updating:
Colocations handled automatically by placer.


2019-12-16 12:34:08,902 : WARNING : From /home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/ops/resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Train on 2000000 samples, validate on 500000 samples
Instructions for updating:
Use tf.cast instead.


2019-12-16 12:34:09,115 : WARNING : From /home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Epoch 1/50
1999744/2000000 [============================>.] - ETA: 0s - loss: 0.4185 - acc: 0.7998
Epoch 00001: val_acc improved from -inf to 0.80707, saving model to models/weights_3_layers_of_size_40_big
2000000/2000000 [==============================] - 31s 16us/sample - loss: 0.4185 - acc: 0.7998 - val_loss: 0.4067 - val_acc: 0.8071
Epoch 2/50
1997184/2000000 [============================>.] - ETA: 0s - loss: 0.4020 - acc: 0.8095
Epoch 00002: val_acc improved from 0.80707 to 0.80948, saving model to models/weights_3_layers_of_size_40_big
2000000/2000000 [==============================] - 33s 16us/sample - loss: 0.4020 - acc: 0.8095 - val_loss: 0.4018 - val_acc: 0.8095
Epoch 3/50
1999616/2000000 [============================>.] - ETA: 0s - loss: 0.3962 - acc: 0.8125
Epoch 00003: val_acc improved from 0.80948 to 0.81208, saving model to models/weights_3_layers_of_size_40_big
2000000/2000000 [==============================] - 31s 15us/sample - loss: 0.3962 - acc: 0.8125 - val_loss: 0.

Epoch 28/50
1999104/2000000 [============================>.] - ETA: 0s - loss: 0.3801 - acc: 0.8211
Epoch 00028: val_acc did not improve from 0.81702
2000000/2000000 [==============================] - 30s 15us/sample - loss: 0.3802 - acc: 0.8211 - val_loss: 0.3903 - val_acc: 0.8156
Epoch 29/50
1996544/2000000 [============================>.] - ETA: 0s - loss: 0.3800 - acc: 0.8212
Epoch 00029: val_acc did not improve from 0.81702
2000000/2000000 [==============================] - 30s 15us/sample - loss: 0.3801 - acc: 0.8212 - val_loss: 0.3899 - val_acc: 0.8160
Epoch 30/50
1996928/2000000 [============================>.] - ETA: 0s - loss: 0.3800 - acc: 0.8212
Epoch 00030: val_acc did not improve from 0.81702
2000000/2000000 [==============================] - 30s 15us/sample - loss: 0.3800 - acc: 0.8213 - val_loss: 0.3929 - val_acc: 0.8148
Epoch 31/50
1996800/2000000 [============================>.] - ETA: 0s - loss: 0.3798 - acc: 0.8213
Epoch 00031: val_acc did not improve from 0.81702
2

Epoch 6/50
1997312/2000000 [============================>.] - ETA: 0s - loss: 0.3877 - acc: 0.8173
Epoch 00006: val_acc improved from 0.81406 to 0.81504, saving model to models/weights_3_layers_of_size_45_big
2000000/2000000 [==============================] - 30s 15us/sample - loss: 0.3877 - acc: 0.8173 - val_loss: 0.3920 - val_acc: 0.8150
Epoch 7/50
1998976/2000000 [============================>.] - ETA: 0s - loss: 0.3863 - acc: 0.8180
Epoch 00007: val_acc did not improve from 0.81504
2000000/2000000 [==============================] - 30s 15us/sample - loss: 0.3863 - acc: 0.8180 - val_loss: 0.3940 - val_acc: 0.8135
Epoch 8/50
1997568/2000000 [============================>.] - ETA: 0s - loss: 0.3853 - acc: 0.8188
Epoch 00008: val_acc did not improve from 0.81504
2000000/2000000 [==============================] - 30s 15us/sample - loss: 0.3852 - acc: 0.8188 - val_loss: 0.3939 - val_acc: 0.8142
Epoch 9/50
1997312/2000000 [============================>.] - ETA: 0s - loss: 0.3844 - acc: 0.

1997312/2000000 [============================>.] - ETA: 0s - loss: 0.3772 - acc: 0.8228
Epoch 00033: val_acc did not improve from 0.81726
2000000/2000000 [==============================] - 30s 15us/sample - loss: 0.3772 - acc: 0.8228 - val_loss: 0.3894 - val_acc: 0.8163
Epoch 34/50
1997056/2000000 [============================>.] - ETA: 0s - loss: 0.3771 - acc: 0.8230
Epoch 00034: val_acc did not improve from 0.81726
2000000/2000000 [==============================] - 30s 15us/sample - loss: 0.3771 - acc: 0.8230 - val_loss: 0.3890 - val_acc: 0.8169
Epoch 35/50
1997312/2000000 [============================>.] - ETA: 0s - loss: 0.3770 - acc: 0.8229
Epoch 00035: val_acc did not improve from 0.81726
2000000/2000000 [==============================] - 30s 15us/sample - loss: 0.3770 - acc: 0.8229 - val_loss: 0.3893 - val_acc: 0.8164
Epoch 36/50
1999104/2000000 [============================>.] - ETA: 0s - loss: 0.3769 - acc: 0.8229
Epoch 00036: val_acc did not improve from 0.81726
2000000/20000

1997824/2000000 [============================>.] - ETA: 0s - loss: 0.3811 - acc: 0.8208
Epoch 00010: val_acc improved from 0.81600 to 0.81607, saving model to models/weights_3_layers_of_size_50_big
2000000/2000000 [==============================] - 30s 15us/sample - loss: 0.3811 - acc: 0.8208 - val_loss: 0.3890 - val_acc: 0.8161
Epoch 11/50
1996416/2000000 [============================>.] - ETA: 0s - loss: 0.3804 - acc: 0.8210
Epoch 00011: val_acc improved from 0.81607 to 0.81662, saving model to models/weights_3_layers_of_size_50_big
2000000/2000000 [==============================] - 31s 15us/sample - loss: 0.3804 - acc: 0.8210 - val_loss: 0.3883 - val_acc: 0.8166
Epoch 12/50
1997440/2000000 [============================>.] - ETA: 0s - loss: 0.3798 - acc: 0.8214
Epoch 00012: val_acc improved from 0.81662 to 0.81685, saving model to models/weights_3_layers_of_size_50_big
2000000/2000000 [==============================] - 31s 16us/sample - loss: 0.3798 - acc: 0.8214 - val_loss: 0.3895 -

KeyboardInterrupt: 

## To execute this after having trained the above model

After having decided which one is the best model to use, just load the correct model, set the parameters of the model_star so that it matches the one of the best model, and compute the results!

(The below part needs to be rewritten to take advantage of the python modules)

In [ ]:
# Recreate exactl the best model
model_star = keras.Sequential([
        keras.layers.InputLayer(input_shape=(sentence_train_x.shape[1],)),   # the input shape is the number of words in the bow dictionary
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(2, activation='softmax')   # Only 0 and 1
    ])

model_star.load_weights('models/weights_2_layers_100_big')

model_star.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
print(model_star)

In [ ]:
# Test it on the test dataset to see if it gives the same best results.
model_star.evaluate(sentence_test_x,  sentence_test_y, verbose=2)[1]
model_star.predict(sentence_test_x)

## GET the test data and predict them

In [ ]:
test_df_cleaned = pd.read_pickle("dataframes/test_cleaned.pickle")

In [ ]:
test_df_cleaned.head()

In [ ]:
sentence_submission_x = create_sentence_vectors_submission(test_df_cleaned['sentence'], word_vector_size, w2v_model)

In [ ]:
sentence_submission_x.shape

In [ ]:
model_star.predict(sentence_submission_x)

In [ ]:
predictions = []
for el in model_star.predict(sentence_submission_x):
    predictions.append(-1 if el[0] > el[1] else 1)

print(predictions[:10])

results = pd.DataFrame({
    "Id": test_df_cleaned['label'],
    "Prediction": predictions
})

results.head(20)

In [ ]:
results.to_csv('Submission.csv', index=False)